In [ ]:
import subprocess
import sys
import re
import time

# outputfileheader=sys.argv[1]

# ls_output = subprocess.check_output('gunzip fastq/*.gz',shell=True)
outputfileheader='RSIX'
dir='/Volumes/smolke-lab$/Joy/MFS/RSIX/fastq2csv'

R1files = subprocess.check_output('ls '+dir+'/*_R1*',shell=True)
R2files = subprocess.check_output('ls '+dir+'/*_R2*',shell=True)

R1joined=''.join(R1files)
R1list=R1joined.split('\n')

R2joined=''.join(R2files)
R2list=R2joined.split('\n')

hasindexing=False
pat=re.compile('([0-9]+)')


print R1list

In [ ]:
# Join the two read end sequences together for paired-end reads that do overlap
f=open('pearallfrompy.sh','a')

if hasindexing:
    for i in range(len(R1list)):
        cmdstr= '~/Documents/Bioinformatics/PEAR/src/pear -f '+R1list[i]+' -r '+R2list[i]+' -o '+outputfileheader+'_I'+Inum+' > '+outputfileheader+'_I'+Inum+'_peared.log'
        f.write(cmdstr+'\n')
    f.close()
else:
    cmdstr= '~/Documents/Bioinformatics/PEAR/src/pear -f '+R1list[0]+' -r '+R2list[0]+' -o '+outputfileheader+' > '+outputfileheader+'_peared.log'
    f.write(cmdstr+'\n')
    f.close()

subprocess.call('chmod +x pearallfrompy.sh',shell=True)
subprocess.call('./pearallfrompy.sh',shell=True)

# remove unique coverage IDs
# for the pJX20-sTRSVctllib
start = time.time()

cmdstr1='perl -n -e \''
cmdstr2='/([A-Z]+GGATCC)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(AAACAAACAAA[A-Z]+-?[A-Z]+AAAAAGAAAAATAAAAA)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(GGTACC[A-Z]+)/ && print "$1$2$3$4$5\\n"\' ' 
cmdstr3=outputfileheader+'*.assembled.fastq > '+outputfileheader+'_ctrlseqs.seq'
print cmdstr1+cmdstr2+cmdstr3
subprocess.call(cmdstr1+cmdstr2+cmdstr3,shell=True)


end = time.time()
print(end - start)



In [ ]:
# Join the two read end sequences together for paired-end reads that do NOT overlap
# this can take a very long time, if the files are big (>a few GB)
import time
start = time.time()

f=open('pasteallfrompy.sh','a')

for i in range(len(R1list)):
    m=re.search(pat,R1list[i])
    if hasindexing:
        if m:
            Inum=m.group(2)
            cmdstr='paste -d "-" '+outputfileheader+'_I'+Inum+'.unassembled.forward.fastq '+outputfileheader+'_I'+Inum+'.unassembled.reverse.fastq > '+outputfileheader+'_I'+Inum+'.unassembled.pasted.fastq'
    else:
        cmdstr='paste -d "-" '+outputfileheader+'.unassembled.forward.fastq '+outputfileheader+'.unassembled.reverse.fastq > '+outputfileheader+'.unassembled.pasted.fastq'
#         print cmdstr
        
    f.write(cmdstr+'\n')
        
f.close()
# subprocess.call('chmod +x pasteallfrompy.sh',shell=True)
# subprocess.call('./pasteallfrompy.sh',shell=True)

end = time.time()
print(end - start)

# grep all the pasted files that contain cdGII aptamer, and join them together
# GCTGTCACCGAGAAACTGTGAAGTATATCTTAAACCTGGGCACTT AAAAGATATATGGAGTTAGTAGTGCAACCTGCTCGGTCTGATGA GTCCNNNNNNGGACGAAACAGC

cmdstr1='perl -n -e \''
cmdstr2='/(^[A-Z]+GGATCC)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(AAACAAACAAAGCTGTCACCGAGAAACTGTGAAGTATATCTTAAACCTGGGCACTT)([A-Z]+-?[A-Z]+)(GTCC[A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T]?GGACGAAACAGCAAAAAGAAAAATAAAAA)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T]GGTACC[A-Z]+)/ && print "$1$2$3AAAAGATATATGGAGTTAGTAGTGCAACCTGCTCGGTCTGATGA$5$6\\n"\' ' 
cmdstr3=outputfileheader+'.unassembled.pasted.fastq > '+outputfileheader+'_cdGII.seq'
print cmdstr1+cmdstr2+cmdstr3
subprocess.call(cmdstr1+cmdstr2+cmdstr3,shell=True)

# grep all the pasted files that contain cdGI aptamer, and join them together
# GCTGTCACCGCGCACAGGGCAAACCATTCGAAAGAGTGGGACGC AAAGCCTCCGGCCTAAACCAGAAGACATGGTAGGTAGCGGGGTTACCGCGGTCTGATGA GTCCNNNNNNGGACGAAACAGC

cmdstr1='perl -n -e \''
cmdstr2='/(^[A-Z]+GGATCC)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(AAACAAACAAAGCTGTCACCGCGCACAGGGCAAACCATTCGAAAGAGTGGGACGC)([A-Z]+-?[A-Z]+)(GTCC[A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T]?GGACGAAACAGCAAAAAGAAAAATAAAAA)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T]GGTACC[A-Z]+)/ && print "$1$2$3AAAGCCTCCGGCCTAAACCAGAAGACATGGTAGGTAGCGGGGTTACCGCGGTCTGATGA$5$6\\n"\' ' 
cmdstr3=outputfileheader+'.unassembled.pasted.fastq > '+outputfileheader+'_cdGI.seq'
print cmdstr1+cmdstr2+cmdstr3
subprocess.call(cmdstr1+cmdstr2+cmdstr3,shell=True)





In [ ]:
# combine the three different libraries into one file, _allseqs.seq

seqfiles = subprocess.check_output('ls *_*.seq',shell=True)

seqjoined=''.join(seqfiles)
seqlist=seqjoined.split('\n')

seqstr=' '.join(seqlist)

cmdstr='cat '+seqstr+'> '+outputfileheader+'_allseqs.seq'
print cmdstr

subprocess.call(cmdstr,shell=True)

# remove unique coverage IDs
start = time.time()

cmdstr1='perl -n -e \''
cmdstr2='/([A-Z]+GGATCC)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(AAACAAACAAA[A-Z]+-?[A-Z]+AAAAAGAAAAATAAAAA)([A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T][A|C|G|T])(GGTACC[A-Z]+)/ && print "$1$3$5\\n"\' ' 
cmdstr3=outputfileheader+'_allseqs.seq > '+outputfileheader+'_all.seq'
print cmdstr1+cmdstr2+cmdstr3
subprocess.call(cmdstr1+cmdstr2+cmdstr3,shell=True)
end = time.time()
print(end - start)

In [ ]:
# demultiplex / assign barcodes

%run -i parseBarcodes.py barcodes.txt RSIX_all.seq RSIX

In [ ]:
# convert demultiplexed file into .csv file
import pandas as pd
import numpy as np
from subprocess import call

start = time.time()
libindeces=range(0,17)

bcfile=outputfileheader+'_barcoded.txt'
text_file = open(bcfile, "r")
seqs = text_file.readlines()
text_file.close()


seqval=[]
seqcnt=[]


for x in seqs:
    s,c=np.unique(x.split(),return_counts=True)
    seqval.append([s])
    seqcnt.append([c])

frames=[]
for i in range(len(seqcnt)):
    tsc=tuple(seqcnt[i][0])
    df = pd.DataFrame(data=list(tsc),columns=[libindeces[i]],index=(tuple(seqval[i][0])))
    frames.append(df)

combined=pd.concat(frames,axis=1)
combinedcounts=combined.reindex_axis(sorted(combined.columns), axis=1)
combinedcounts.to_csv(outputfileheader+'_combinedcounts.csv', index=True, header=False,columns=range(0,len(combined.columns)+1))
combinedcounts

end = time.time()
print(end - start)


In [ ]:
print combinedcounts.sum()